In [14]:
import pandas as pd
import numpy as np
import plotly.graph_objects as go
import plotly.express as px
from datetime import datetime
from datetime import timedelta
import tensorflow as tf

In [15]:
Data = pd.read_csv('Apple Stocks.csv', parse_dates=['Date'])
Data['date'] = Data[['Date']]
Data['close'] = Data['Close']
Data.drop(columns=['Date','Close'],inplace=True)
df = Data.copy()

In [16]:
df

,Open,High,Low,Adj Close,Volume,date,close
0,0.128348,0.128906,0.128348,0.100178,469033600,1980-12-12,0.128348
1,0.122210,0.122210,0.121652,0.094952,175884800,1980-12-15,0.121652
2,0.113281,0.113281,0.112723,0.087983,105728000,1980-12-16,0.112723
3,0.115513,0.116071,0.115513,0.090160,86441600,1980-12-17,0.115513
4,0.118862,0.119420,0.118862,0.092774,73449600,1980-12-18,0.118862
...,...,...,...,...,...,...,...
10463,132.869995,135.199997,131.440002,131.880005,122207100,2022-06-13,131.880005
10464,133.130005,133.889999,131.479996,132.759995,84784300,2022-06-14,132.759995
10465,134.289993,137.339996,132.160004,135.429993,91533000,2022-06-15,135.429993
10466,132.080002,132.389999,129.039993,130.059998,108123900,2022-06-16,130.059998


In [17]:
fig = px.line( x = Data['date'],y=Data['close'] )
fig.show()

In [18]:
df_date = df[['date']]
df_close = df[['close']]
df.drop(columns=['date'],inplace=True)
df.head()

,Open,High,Low,Adj Close,Volume,close
0,0.128348,0.128906,0.128348,0.100178,469033600,0.128348
1,0.122210,0.122210,0.121652,0.094952,175884800,0.121652
2,0.113281,0.113281,0.112723,0.087983,105728000,0.112723
3,0.115513,0.116071,0.115513,0.090160,86441600,0.115513
4,0.118862,0.119420,0.118862,0.092774,73449600,0.118862


In [19]:
from sklearn.preprocessing import StandardScaler
for feature in list(df.columns.values):
  scaler = StandardScaler()
  scaler.fit( df[[feature]] )
  df[feature] = scaler.transform( df[[feature]] )

In [20]:
def Create_Timestaped_Data_Matrix_Multi( df, size ):
  X,y,dates = [],[],[]
  for i in range(df.shape[0]-size):
    X.append( np.asarray(df.values[i:i+size]).astype(np.float64) )
    y.append( df['close'].values[i+size] )
    dates.append(df_date['date'][i+size])
  return np.array(X), np.array(y), dates

window_size = 50
X,y,dates = Create_Timestaped_Data_Matrix_Multi( df, size=window_size )

In [21]:
X.shape, y.shape

((10418, 50, 6), (10418,))

In [22]:
test_size_ratio = 0.1
train_size = int((1-test_size_ratio)*len(y))
# Train Set
y_train = y[:train_size]
dates_train = dates[:train_size]
y_test = y[train_size:]
X_train = X[:train_size]
# Test Set
dates_test = dates[train_size:]
X_test = X[train_size:]
y_train_original = Data['close'][window_size:train_size+window_size] # Storing original columns for plotting graph
y_test_original = Data['close'][train_size+window_size:] # Storing original columns for plotting graph
# Check length
# len(y_train)+len(y_test),len(y)

In [23]:
fig = go.Figure()
fig.add_trace( go.Scatter( x=dates_train, y=y_train,mode='lines', name="Train Set" ) )
fig.add_trace( go.Scatter( x=dates_test, y=y_test,mode='lines', name="Test Set" ) )
fig.show()

In [24]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dropout
model = Sequential()
model.add( LSTM( 500, return_sequences =True, input_shape=(50,6) ) )
model.add( LSTM(100) )
model.add( Dense(1) )
model.compile( loss='mean_squared_error', optimizer='adam' )
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_2 (LSTM)               (None, 50, 500)           1014000   
                                                                 
 lstm_3 (LSTM)               (None, 100)               240400    
                                                                 
 dense_1 (Dense)             (None, 1)                 101       
                                                                 
Total params: 1254501 (4.79 MB)
Trainable params: 1254501 (4.79 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [ ]:
history = model.fit(X_train,y_train,validation_data = (X_test,y_test), epochs=20 , verbose=1)

Epoch 1/20
293/293 [==============================] - 153s 505ms/step - loss: 0.0013 - val_loss: 0.6772
Epoch 2/20
293/293 [==============================] - 140s 477ms/step - loss: 1.5532e-04 - val_loss: 0.3377
Epoch 3/20
293/293 [==============================] - 141s 483ms/step - loss: 1.3098e-04 - val_loss: 0.2080
Epoch 4/20
293/293 [==============================] - 140s 477ms/step - loss: 1.1639e-04 - val_loss: 0.1731
Epoch 5/20
293/293 [==============================] - 140s 478ms/step - loss: 1.0220e-04 - val_loss: 0.1668
Epoch 6/20
293/293 [==============================] - 140s 479ms/step - loss: 9.6466e-05 - val_loss: 0.1398
Epoch 7/20
293/293 [==============================] - 141s 482ms/step - loss: 9.0486e-05 - val_loss: 0.0898
Epoch 8/20
293/293 [==============================] - 502s 2s/step - loss: 7.7031e-05 - val_loss: 0.1830
Epoch 9/20
293/293 [==============================] - 134s 457ms/step - loss: 9.1229e-05 - val_loss: 0.1772
Epoch 10/20
293/293 [==============

In [ ]:
def changing_shape( array ):
  temp = []  
  for i in range(len(array)):
    temp.append(array[i][0])
  return temp

In [ ]:
def Prediction_Inverse_Scaling_Reshaping( model, X_train, X_test ):
  # Predicting values and Appyling Inverse Scaler
  y_pred = model.predict( X_test )
  y_pred = scaler.inverse_transform( y_pred )

  y_pred_train = model.predict( X_train )
  y_pred_train = scaler.inverse_transform( y_pred_train )

  # Reshaping
  y_pred = changing_shape(y_pred)
  y_pred_train = changing_shape( y_pred_train )

  return y_pred, y_pred_train


y_pred, y_pred_train = Prediction_Inverse_Scaling_Reshaping( model, X_train, X_test )

In [ ]:
fig = go.Figure()
fig.add_trace( go.Scatter( x=dates_train, y=y_train_original,mode='lines', name="Original Train Set" ) )
fig.add_trace( go.Scatter( x=dates_test, y=y_test_original,mode='lines', name="Original Test Set" ) )
fig.add_trace( go.Scatter( x=dates_train, y=y_pred_train, name="Predicted Train Set" ) )
fig.add_trace( go.Scatter( x=dates_test, y=y_pred,mode='lines', name="Predicted Test Set" ) )
fig.show()

In [ ]:
from sklearn.metrics import mean_squared_error
np.sqrt( mean_squared_error( y_pred, y_test_original ) )

In [ ]:
# Extracting last values values (of certain window size) from the df to use it for predicting stock price for next date
last = df.iloc[-window_size:].values
pred_close = model.predict( last.reshape( 1, window_size, 6 ) )[0][0]
pred_close = scaler.inverse_transform( pred_close.reshape(-1,1) )[0][0]

# Adding one day to last date ( for adding "next" day date )
temp_date = dates_test[-1] + timedelta(days=1)

# Creating dataframe of 'next' and prediction on that day then appending them to df
future_pred_df =pd.DataFrame({'date':[temp_date],'close':[pred_close]})

In [ ]:
future_pred_df